In [12]:
import pandas as pd
import dataframe_image as dfi # for saving styled data frame print-out table as png

pd.set_option('display.max_columns', 15)
pd.set_option('display.max_rows', 50)
pd.options.display.float_format = '{:,.4f}'.format

## Analyze Top WBTC-WETH Uniswap V3 Pools 

This table of [Top Pools](https://revert.finance/#/top-positions?sort=underlying_value&underlying-value-from=20000&apr-from=20&token1=0xc02aaa39b223fe8d0a0e5c4f27ead9083c756cc2&token0=0x2260fac5e5542a773aa44fbcfedf7c193bc2c599&age-from=14)
was scraped from revert finance. Check out the [scraping script](https://github.com/coindataschool/mixture/blob/main/notebook/univ3_wbtc_weth_top_positions_01_scrape_data.ipynb).

In [13]:
df = pd.read_csv('../data/univ3_wbtc_weth_lp_top_positions.csv')\
       .drop(columns=['Unnamed: 0', 'url'])
df['APR'] = df.APR/100
df['fee_APR'] = df.fee_APR/100
df

,pool_fees,PnL,APR,fee_APR,value,age,price_lwr,price_upr
0,0.30%,"85,084.1400",0.2030,0.2039,"2,804,470.2800",54.5000,11.5061,15.1631
1,0.30%,"142,183.5900",0.6714,0.3743,"822,076.0200",76.1000,12.1445,13.6928
2,0.30%,"40,051.5300",0.2746,0.2754,"814,482.3700",69.5000,13.6108,15.9086
3,0.30%,"37,183.8500",0.2396,0.2399,"756,293.9600",74.9000,13.0511,15.3461
4,0.30%,"162,142.6400",0.2068,0.2095,"731,351.5100",460.3000,12.1445,16.1975
5,0.30%,"33,903.5000",0.2386,0.2396,"692,087.8000",74.9000,13.0511,15.3461
6,0.30%,"12,674.2100",0.8756,0.9174,"657,723.0300",15.2000,12.3650,14.5394
7,0.30%,"19,732.6900",0.2082,0.2096,"629,284.5000",55.0000,11.7151,15.6248
8,0.30%,"18,400.4800",0.2065,0.2143,"463,959.9000",87.0000,11.3008,15.2543
9,0.30%,"22,634.5900",0.2063,0.2599,"356,716.9300",110.4000,12.0718,15.9086


In [14]:
df['pool_fees'].value_counts()

0.30%    25
0.05%     6
Name: pool_fees, dtype: int64

### Age & APR by Pool Type

In [15]:
ha = df.groupby('pool_fees')[['age', 'fee_APR', 'APR']].aggregate(['min']) 
dfi.export(ha, '../png/min_age_apr_by_pool_type.png')

[1115/092641.708422:INFO:headless_shell.cc(657)] Written to file /var/folders/qq/v47zfw7s1kn38gw67q4v194r0000gn/T/tmptsqvj1w1/temp.png.


In [16]:
ha = df.groupby('pool_fees')[['age', 'fee_APR', 'APR']].aggregate(['median'])
dfi.export(ha, '../png/med_age_apr_by_pool_type.png')

[1115/092642.508800:INFO:headless_shell.cc(657)] Written to file /var/folders/qq/v47zfw7s1kn38gw67q4v194r0000gn/T/tmpzdz5fwt1/temp.png.


In [17]:
ha = df.groupby('pool_fees')[['age', 'fee_APR', 'APR']].aggregate(['max'])
dfi.export(ha, '../png/max_age_apr_by_pool_type.png')

[1115/092643.257462:INFO:headless_shell.cc(657)] Written to file /var/folders/qq/v47zfw7s1kn38gw67q4v194r0000gn/T/tmp2_k8zgqa/temp.png.


### Price Range by Pool Type

In [18]:
ha = df.groupby('pool_fees')[['price_lwr', 'price_upr']].aggregate(['min', 'median', 'max'])
dfi.export(ha, '../png/price_rng_by_pool_type.png')

[1115/092644.006601:INFO:headless_shell.cc(657)] Written to file /var/folders/qq/v47zfw7s1kn38gw67q4v194r0000gn/T/tmprng2m0ys/temp.png.
